In [1]:
import pandas as pd
import numpy as np
from mysql.connector import connect
import json
from os import path
from utils import get_db_creds
import sqlalchemy

In [2]:
db_creds = get_db_creds()
connection = connect(**db_creds)

In [4]:
conn = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'
            .format(db_creds["user"], db_creds["password"], db_creds["host"], db_creds["database"])).connect()

In [3]:
users_df = pd.read_csv("users.csv")
users_df = users_df.rename(columns={"hashed_password": "password"})
users_df["username"] = users_df["username"].str.replace(" ", "_")
username_set = set()
for i in range(len(users_df)):
    counter = 1
    while (users_df.iloc[i]["username"] in username_set):
        users_df.loc[users_df.index == i, "username"] = users_df.iloc[i]["username"] + "_" + str(counter)
        counter += 1
    username_set.add(users_df.iloc[i]["username"])
users_df.head()

,user_id,username,password
0,1,Elizebeth_Marshall,password
1,2,Dwayne_Stpierre,password
2,3,Kim_Mclaughlin,password
3,4,Frank_Barrette,password
4,5,Timothy_Hackman,password


In [5]:
users_df.to_sql(name="User", con=conn, if_exists="append", index=False,)

In [6]:
cuisines_df = pd.read_csv("cuisines.csv")
cuisines_df.head()

,cuisine_id,cuisine_name
0,1,north-american
1,2,global
2,3,asian
3,4,german
4,5,italian


In [7]:
cuisines_df.to_sql(name="Cuisine", con=conn, if_exists="append", index=False,)

In [8]:
tags_df = pd.read_csv("tags.csv")
tags_df = tags_df.fillna("")
tags_df.head()
tag_id_to_tag_map = {i+1: cuisine_name for i, cuisine_name in tags_df.to_dict()["0"].items()}
tags_df = pd.DataFrame.from_dict(tag_id_to_tag_map, orient="index")
tags_df["tag_id"] = tags_df.index
tags_df = tags_df.rename(columns={0: "tag_text"})
tags_df.head()

,tag_text,tag_id
1,,1
2,cuisine,2
3,creole,3
4,lunch,4
5,poultry,5


In [9]:
tags_df.to_sql(name="Tag", con=conn, if_exists="append", index=False,)

In [72]:
ingr_map_df = pd.read_csv("ingr_map.csv")
ingr_map_df = ingr_map_df.rename(columns={"replaced": "ingredient_name", "id": "ingredient_id"})
ingr_map_df = ingr_map_df[["ingredient_name", "ingredient_id"]]
ingr_map_df = ingr_map_df.drop_duplicates()
ingr_map_df.loc[ingr_map_df["ingredient_id"] == 0, "ingredient_id"] = ingr_map_df["ingredient_id"].max() + 1
ingr_map_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8023 entries, 0 to 11658
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ingredient_name  8023 non-null   object
 1   ingredient_id    8023 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 188.0+ KB


In [73]:
ingr_map_df.to_sql(name="Ingredient", con=conn, if_exists="append", index=False,)

In [77]:
recipe_df = pd.read_csv("recipe_155k_with_all_updated_columns.csv")
columns_needed = ["recipe_id", "creator_id", "recipe_name", "serves",
                 "date_submitted", "cuisine_id", "description",
                 "recipe_text", "calories", "time_to_prepare",
                 "img_url"]
recipe_df = recipe_df[columns_needed]
recipe_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155751 entries, 0 to 155750
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   recipe_id        155751 non-null  int64  
 1   creator_id       155751 non-null  int64  
 2   recipe_name      155751 non-null  object 
 3   serves           155751 non-null  int64  
 4   date_submitted   155751 non-null  object 
 5   cuisine_id       155751 non-null  int64  
 6   description      152310 non-null  object 
 7   recipe_text      155751 non-null  object 
 8   calories         155751 non-null  float64
 9   time_to_prepare  155751 non-null  int64  
 10  img_url          155751 non-null  object 
dtypes: float64(1), int64(5), object(5)
memory usage: 13.1+ MB


In [20]:
recipe_df.to_sql(name="Recipe", con=conn, if_exists="append", index=False,)

In [74]:
recipe_ingredients_df = pd.read_csv("recipe_ingredients_df.csv")
recipe_ingredients_df = recipe_ingredients_df.drop_duplicates(subset=["recipe_id", "ingredient_id"])
recipe_ingredients_df.loc[recipe_ingredients_df["ingredient_id"] == 0, "ingredient_id"] = ingr_map_df["ingredient_id"].max()
recipe_ingredients_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1364897 entries, 0 to 1366603
Data columns (total 4 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   recipe_id         1364897 non-null  int64  
 1   ingredient_id     1364897 non-null  int64  
 2   quantity          1279760 non-null  float64
 3   measurement_unit  1279760 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 52.1+ MB


In [76]:
recipe_ingredients_df.to_sql(name="RecipeIngredient", con=conn, if_exists="append", index=False,)

In [79]:
interactions_df = pd.read_csv("interactions_df.csv")
interactions_df.to_sql(name="Interaction", con=conn, if_exists="append", index=False,)

IntegrityError: (mysql.connector.errors.IntegrityError) 1062 (23000): Duplicate entry '294-27208' for key 'Interaction.PRIMARY'
[SQL: INSERT INTO `Interaction` (user_id, recipe_id, interaction_date, rating, review) VALUES (%(user_id)s, %(recipe_id)s, %(interaction_date)s, %(rating)s, %(review)s)]
[parameters: ({'user_id': 899, 'recipe_id': 52469, 'interaction_date': '2003-04-24', 'rating': 5, 'review': "It's GENIUS! I love this recipe- thank you so much- it was a big hit! I'll definitely use this one more often!"}, {'user_id': 9527, 'recipe_id': 241877, 'interaction_date': '2009-02-20', 'rating': 4, 'review': "Good side dish.  I only made half a recipe and I had to cook it longer than stated but good.  Didn't add cheese but may next time."}, {'user_id': 12460, 'recipe_id': 373493, 'interaction_date': '2012-07-07', 'rating': 5, 'review': 'Delicious! I did eyeball the honeydew (used a whole honeydew, instead of measuring, but it was close) and used plain yogurt instead of vanilla, but o 

In [30]:
users_df.to_csv("users.csv", index=False)

In [31]:
raw_interactions_df.to_csv("interactions_with_updated_users.csv", index=False)

In [32]:
recipe_df.to_csv("recipe_df_178k_with_updated_users.csv", index=False)

In [70]:
# KEEP iloc[292740] for recipe_ingredients_df in mind
recipe_ingredients_df.iloc[292740]

recipe_id                   118843
ingredient_id                    0
quantity                       1.0
measurement_unit    (12 ounce) box
Name: 293131, dtype: object

In [71]:
ingr_map_df[ingr_map_df["ingredient_id"] == 0]

,ingredient_name,ingredient_id
3483,'s baking chocolate,0
